In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config Completer.use_jedi = False
import xarray as xr
import warnings
from netCDF4 import Dataset
# creating a series of datatime 
from datetime import date, datetime, timedelta
# plotting time series data 
import seaborn as sns

from pandas import datetime
from matplotlib import pyplot

In [2]:
from astral import LocationInfo
from astral.sun import sun

In [3]:
year = 2055
year2=year+1

#### Determinig the the closest point to Nantes coordinates

In [4]:
## Visualize netCDF file of temperature  
tas1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\tas_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010100-'+str(year)+'12312200.nc')
print("temperature data "+" : " + str(tas1.variables.keys()))

temperature data  : dict_keys(['time', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'height', 'tas'])


In [5]:
# Storing the coordinates (lon, lat) into variables 
lats = tas1.variables['rlat'][:]
lons = tas1.variables['rlon'][:]
# Assigned location coordinates 
lat_Nantes =  -1.7927858546021267    # 47.210517 ; latitude coordinates of Centre ville in rotated pole grid 
lon_Nantes =  -13.146332621887227    #-1.546285  ; longitude coordinates of Centre ville in rotated pole grid 
# Selecting the nearest grid point to assigned location
# storing the squared differences of 
sq_diff_lat = (lats-lat_Nantes)**2
sq_diff_lon = (lons-lon_Nantes)**2

## Verifying the shape of coordinates 
print("Total number of grid points along y axis(lat)"+" = " + str(len(sq_diff_lat)))
print("Total number of grid points along x axis(lon)"+" = " + str(len(sq_diff_lon)))

## Identifying the index of minimum values for lat and lon sq_differences
min_index_lat = sq_diff_lat.argmin()
min_index_lon = sq_diff_lon.argmin()

print("Index of closest data point along y axis(lat) to assigned location"+" = " + str(min_index_lat))
print("Index of closest data point along x axis(lon) to assigned location"+" = " + str(min_index_lon))

Total number of grid points along y axis(lat) = 412
Total number of grid points along x axis(lon) = 424
Index of closest data point along y axis(lat) to assigned location = 196
Index of closest data point along x axis(lon) to assigned location = 138


#### Creating a date-time dataframe

In [6]:
# creating a DataFrame of year, month, day and hours 
def datetime_range(start,end,delta):
    current=start
    if not isinstance(delta,timedelta):
        delta = timedelta(**delta)
    while current < end:
        yield current
        current +=delta

In [7]:
# Dataframe of time and dates: each subsequent row difference is 3 hours
start = datetime(year,0o1,0o1)
end = datetime(year2,0o1,0o1)

df = pd.DataFrame({'DateTime':datetime_range(start,end,{'days':0,'hours':3})})

# Removing leap day data if year is a leap year 
df = df[~((df['DateTime'].dt.month == 2) & (df['DateTime'].dt.day == 29))]
# reindex in place
df= df.reset_index()
## Drop index column 
df=df.drop(['index'], axis=1)
# Visualize beginning and end of dataframe
print(df.head(2))
print(df.tail(1))

             DateTime
0 2055-01-01 00:00:00
1 2055-01-01 03:00:00
                DateTime
2919 2055-12-31 21:00:00


#### Reading NetCDF files for 6 climate variables of the closest grid point to the assigned location

In [8]:
# Visualize netCDF file of temperature  
tas1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\tas_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010100-'+str(year)+'12312200.nc')
print("temperature data "+" : " + str(tas1.variables.keys()))

# Visualize and load humidity ratio data "%" 
hurs1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\hurs_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010100-'+str(year)+'12312200.nc')
print("humidity ratio data "+" : " + str(hurs1.variables.keys()))

# Visualize and load Surface Downwelling Shortwave Radiation :units = "W m-2"; data"
rsds1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\rsds_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010130-'+str(year)+'12312230.nc')
print("Surface Downwelling Shortwave Radiation data "+" : " + str(rsds1.variables.keys()))

# Visualize and load Near surface wind speed data :units = "m s-1"; data"
sfcWind1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010100-'+str(year)+'12312200.nc')
print("Near surface wind speed data "+" : " + str(sfcWind1.variables.keys()))

# Visualize and load total cloud cover fraction :units "%"
clt1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\clt_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010130-'+str(year)+'12312230.nc')
print("total cloud cover fraction data "+" : " + str(clt1.variables.keys()))

# Visualize and load atmospheric pressure :units "%"
ps1 = Dataset(r'F:\Donnees_PC\Weather_files\CORDEX_weather_data\MPI_REMO\MPI_REMO2055\ps_EUR-11_MPI-M-MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_3hr_'+str(year)+'01010100-'+str(year)+'12312200.nc')
print("surface_air_pressure data Pa "+" : " + str(ps1.variables.keys()))

temperature data  : dict_keys(['time', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'height', 'tas'])
humidity ratio data  : dict_keys(['time', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'height', 'hurs'])
Surface Downwelling Shortwave Radiation data  : dict_keys(['time', 'time_bnds', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'rsds'])
Near surface wind speed data  : dict_keys(['time', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'height', 'sfcWind'])
total cloud cover fraction data  : dict_keys(['time', 'time_bnds', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'clt'])
surface_air_pressure data Pa  : dict_keys(['time', 'rlat', 'rlon', 'rotated_latitude_longitude', 'lat', 'lon', 'lat_vertices', 'lon_vertices', 'ps'])


#### Appending climate variable values of the closest grid point to the Date-time dataframe

In [9]:
temp = tas1.variables['tas']
humidity = hurs1.variables['hurs']
radiation = rsds1.variables['rsds']
Wind = sfcWind1.variables['sfcWind']
Cloud_fract = clt1.variables['clt']
s_pressure = ps1.variables['ps']

# Calculating values of the indices that were previouosly found with arg_min
temperat =[]
humidityR =[]
radiationSh =[]
WindS =[]
Cloud =[]
S_pressure =[]

for index, row in df.iterrows():  
    
    a = temp[index,min_index_lat,min_index_lon]-273.15
    b = humidity[index,min_index_lat,min_index_lon]
    c = radiation[index,min_index_lat,min_index_lon]
    d = Wind[index,min_index_lat,min_index_lon]
    e = Cloud_fract[index,min_index_lat,min_index_lon]
    f = s_pressure[index,min_index_lat,min_index_lon]
    
    temperat.append(a)
    humidityR.append(b)
    radiationSh.append(c)
    WindS.append(d)
    Cloud.append(e)
    S_pressure.append(f)
    
df["DB_Temp"] = temperat
df["RelativeHumidity"] = humidityR
df["GHRadiation"] = radiationSh
df["WindSpeed"] = WindS
df["CloudCover"] = Cloud
df["AtmosPressure"] = S_pressure

# Turn objects to float 
df["DB_Temp"] = df.DB_Temp.astype(float)
df["RelativeHumidity"] = df.RelativeHumidity.astype(float)
df["GHRadiation"] = df.GHRadiation.astype(float)
df["WindSpeed"] = df.WindSpeed.astype(float)
df["CloudCover"] = df.CloudCover.astype(float)
df["AtmosPressure"] = df.AtmosPressure.astype(float)

#### Interpolation of 3hr data points to hourly and calculation of remaining fields of weather file.

In [10]:
df=df.set_index('DateTime').resample('H').last()

In [11]:
df['DB_Temp'].interpolate(method='polynomial', order=7, inplace=True)
df['RelativeHumidity'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
df['AtmosPressure'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
df['CloudCover'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)
df['GHRadiation'].interpolate(method='polynomial', order=7, inplace=True)
df['WindSpeed'].interpolate(method='linear', limit_direction='both',limit=2, inplace=True)

#### Estimating dew point temperature

In [12]:
# August–Roche–Magnus formula for dew point temperature approximation: 
def get_dew_point_c(DryBulbTemp, Rel_humidity):
    """Compute the dew point in degrees Celsius
    :param DryBulbTemp: current ambient temperature in degrees Celsius
    :type DryBulbTemp: float
    :param Rel_humidity: relative humidity in %
    :type Rel_humidity: float
    :return: the dew point in degrees Celsius
    :rtype: float
    Set of values originates from the 1974 Psychrometry and Psychrometric Charts, as presented by Paroscientific,[15]
     A = 17.27, B = 237.7 °C; for 0 °C ≤ T ≤ 60 °C (error ±0.4 °C).
    """
    A = 17.27
    B = 237.7
    alpha = ((A * DryBulbTemp) / (B + DryBulbTemp)) + np.log(Rel_humidity/100.0)
    return (B * alpha) / (A - alpha)

In [13]:
df['DewPointTemperature'] = round(get_dew_point_c(df['DB_Temp'],df['RelativeHumidity']),2)

#### Calculating Sunrise and sunset time for the assigned location

In [14]:
# Setting timezone for Nantes city to calculate sunrise and sunset time. Nantes follows Paris timezone 
city = LocationInfo("Nantes","France","Europe/Paris",47.2105, -1.54628)
tz=city.timezone
print(tz)
print(city.latitude)

Europe/Paris
47.2105


In [15]:
import datetime

In [16]:
df=df.reset_index()

In [17]:
# Splitting year, month, day to calculate sunrise and sundown time in each day and add them as a column 
df['year']=df['DateTime'].dt.year
df['month']=df['DateTime'].dt.month
df['day']=df['DateTime'].dt.day

In [18]:
sr = []
st = []
for i, row in df.iterrows(): 
    
    sries=sun(city.observer, date=datetime.date(df.year[i],df.month[i],df.day[i]),tzinfo=city.timezone)["sunrise"]
    sriet=sun(city.observer, date=datetime.date(df.year[i],df.month[i],df.day[i]),tzinfo=city.timezone)["sunset"]
    
    sr.append(sries)
    st.append(sriet)
    
df["sunrise"]=sr
df["sunset"]=st
#print(s["sunset"])

In [19]:
sunsr=df.sunrise[12].tz_localize(None)
sunst=df.sunset[12].tz_localize(None)
hourtime=df.DateTime[12].tz_localize(None)

sunsr=sunsr.strftime("%m/%d/%Y, %H:%M:%S")
sunst=sunst.strftime("%m/%d/%Y, %H:%M:%S")
hourtime=hourtime.strftime("%m/%d/%Y, %H:%M:%S")
                                  
sunrisen = datetime.datetime.strptime(sunsr, '%m/%d/%Y, %H:%M:%S')
sundown = datetime.datetime.strptime(sunst, '%m/%d/%Y, %H:%M:%S')
mytime= datetime.datetime.strptime(hourtime, '%m/%d/%Y, %H:%M:%S')

if mytime<sundown and mytime>sunrisen:
    print(df.GHRadiation[12])
else:
    print(0)

211.78707885742188


In [20]:
# Replacing the interpolated values of GHI that are between sunset and sunrise to zero 
NewGHI=[] #

for i, row in df.iterrows():
    sunsr=df.sunrise[i].tz_localize(None)
    sunst=df.sunset[i].tz_localize(None)
    hourtime=df.DateTime[i].tz_localize(None)

    sunsr=sunsr.strftime("%m/%d/%Y, %H:%M:%S")
    sunst=sunst.strftime("%m/%d/%Y, %H:%M:%S")
    hourtime=hourtime.strftime("%m/%d/%Y, %H:%M:%S")
                                  
    sunrisen = datetime.datetime.strptime(sunsr, '%m/%d/%Y, %H:%M:%S')
    sundown = datetime.datetime.strptime(sunst, '%m/%d/%Y, %H:%M:%S')
    mytime= datetime.datetime.strptime(hourtime, '%m/%d/%Y, %H:%M:%S')

    if mytime<sundown and mytime>sunrisen and df.GHRadiation[i]>0:
        n=df.GHRadiation[i]
    else:
        n=0
        
    NewGHI.append(n)
df["GHI"] = NewGHI

In [21]:
# Drop unnecessary columns for now 
df=df.drop(columns=['GHRadiation','day','sunrise','sunset'])

#### Calcuating DNI using DISC approximation

In [22]:
## Day of year: ordinal days of the year 
newDays=[]
for i, row in df.iterrows():
    DS=df.DateTime[i].tz_localize(None)
    DS=DS.strftime("%m/%d/%Y, %H:%M:%S")
    doy= datetime.datetime.strptime(DS, '%m/%d/%Y, %H:%M:%S').timetuple().tm_yday
    newDays.append(doy)
df['OrdinalDay']= newDays
#tm_hour

In [24]:
import pvlib as pvl
import pytz

In [25]:
## a loop to calculate Solar zenith angle
ZenAngle=[]
for i, row in df.iterrows():
    Zt=df.DateTime[i].tz_localize(None)
    g=pvl.solarposition.get_solarposition(time=Zt,
                                      latitude=47.2105,# latitude of Nantes 
                                      longitude=-1.54628, # Longitude of Nantes 
                                      altitude=None,
                                      pressure=df.AtmosPressure[i],
                                      method='nrel_numpy',
                                      temperature=df.DB_Temp[i]).zenith.values[0]
    ZenAngle.append(round(g,2))
df['SolarZenith']= ZenAngle

In [26]:
import math

In [27]:
# Calculating Direct normal irradiance using zenith angle, ordinal days, atmospheric pressure, and airmass
dni=[]
for i, row in df.iterrows():
    p = pvl.irradiance.disc(ghi=df.GHI[i], 
                      solar_zenith=df.SolarZenith[i], 
                      datetime_or_doy=df.OrdinalDay[i], 
                      pressure=df.AtmosPressure[i], 
                      min_cos_zenith=0.065, 
                      max_zenith=87, 
                      max_airmass=(1/(math.cos(df.SolarZenith[i])+0.50572*(96.07995-df.SolarZenith[i])
                                      **(-1.6364))))['dni']
    # Accounting for cloud cover
    S1=p.tolist()*(1-(df.CloudCover[i])/100)
    dni.append(round(S1,2))
    
df['DNI']= dni

<ipython-input-27-f20766d7981d>:10: RuntimeWarning: invalid value encountered in double_scalars
  max_airmass=(1/(math.cos(df.SolarZenith[i])+0.50572*(96.07995-df.SolarZenith[i])**(-1.6364))))['dni']
C:\Users\obaidullah.yaqubi\AppData\Roaming\Python\Python38\site-packages\pvlib\irradiance.py:1418: RuntimeWarning: overflow encountered in exp
  delta_kn = a + b * np.exp(c*am)


In [28]:
# Seeing how does the date-time dataframe looks like
df.iloc[5000:5004]

,DateTime,DB_Temp,RelativeHumidity,WindSpeed,CloudCover,AtmosPressure,DewPointTemperature,year,month,GHI,OrdinalDay,SolarZenith,DNI
5000,2055-07-28 08:00:00,28.545783,64.471621,2.575628,13.973272,102531.992188,21.18,2055,7,642.757086,209,58.13,0.00
5001,2055-07-28 09:00:00,29.380029,55.186306,2.959718,15.382727,102502.679688,19.44,2055,7,760.595886,209,48.21,0.00
5002,2055-07-28 10:00:00,29.719242,55.577503,3.178748,19.121150,102483.351562,19.86,2055,7,821.576213,209,39.13,746.52
5003,2055-07-28 11:00:00,29.773220,55.968700,3.397777,22.859574,102464.023438,20.03,2055,7,821.657616,209,31.97,549.49


In [29]:
# Verifying how does calculated DNI looks like in the file
p = pvl.irradiance.disc(ghi=df.GHI[5002], 
                      solar_zenith=df.SolarZenith[5002], 
                      datetime_or_doy=df.OrdinalDay[5002], 
                      pressure=df.AtmosPressure[5002], 
                      min_cos_zenith=0.065, 
                      max_zenith=87, 
                      max_airmass=(1/(math.cos(df.SolarZenith[5002])+0.50572*(96.07995-df.SolarZenith[5002])
                                      **(-1.6364))))['dni']
print(round(p.tolist(),2))
# Accounting for cloud cover
S1=p.tolist()*(1-(df.CloudCover[5002])/100)
round(S1,2)

923.01


746.52

In [30]:
colmn = df.columns.tolist()
print (colmn)

['DateTime', 'DB_Temp', 'RelativeHumidity', 'WindSpeed', 'CloudCover', 'AtmosPressure', 'DewPointTemperature', 'year', 'month', 'GHI', 'OrdinalDay', 'SolarZenith', 'DNI']


In [31]:
new_columns = ['DateTime','year','month','DB_Temp','DewPointTemperature','RelativeHumidity',
               'WindSpeed', 'CloudCover','AtmosPressure','GHI','DNI','SolarZenith']
df = df[new_columns]
df.head(2)

,DateTime,year,month,DB_Temp,DewPointTemperature,RelativeHumidity,WindSpeed,CloudCover,AtmosPressure,GHI,DNI,SolarZenith
0,2055-01-01 00:00:00,2055,1,11.928796,10.76,92.573906,4.578446,96.912216,103194.500000,0.0,0.0,155.73
1,2055-01-01 01:00:00,2055,1,8.906792,7.46,90.627139,4.251208,74.319516,103196.804688,0.0,0.0,153.76


#### Calculating diffuse horizontal irradiance

In [36]:
dhi=[]
for i, row in df.iterrows():
    h = df.GHI[i]-(df.DNI[i]*math.cos(df.SolarZenith[i]))
    dhi.append(round(h,2))
    
df['DHI']= dhi

In [35]:
# Removing leap day data if year is a leap year 
df = df[~((df['DateTime'].dt.month == 2) & (df['DateTime'].dt.day == 29))]

In [ ]:
df.to_excel(r'D:\Weather_files\CORDEX_weather_data\MPI_REMO\MPI-ESM-LR_rcp85_r3i1p1_GERICS-REMO2015_v1_'+str(year)+'.xlsx', index = 0)